In [1]:
import torch
from transformers import BartForConditionalGeneration, BartConfig, BartTokenizerFast

In [2]:
model_class = BartForConditionalGeneration
config_class = BartConfig
model_fp = 'facebook/bart-base'

config = config_class.from_pretrained(model_fp)  

model = model_class.from_pretrained(model_fp)
tokenizer = BartTokenizerFast.from_pretrained(model_fp)

In [3]:
example_english_phrase = "Hello world " 
batch = tokenizer(example_english_phrase, return_tensors='pt')

In [4]:
tokenizer.batch_decode(batch['input_ids'])

['<s>Hello world </s>']

In [27]:
def get_prefix_allowed_tokens_fn(allowed_seqs):
    allowed_ids = tokenizer(allowed_seqs)['input_ids']
    allowed_ids = [[2] + allowed_id for allowed_id in allowed_ids]
    print(allowed_ids)
    
    def prefix_constraining_fn(batch_id, input_ids):
        print(batch_id)
        print(input_ids)
        if len(input_ids.size()) == 1:
            prefix_len = len(input_ids)
            prefix = input_ids.tolis
        elif len(input_ids.size()) == 2:
            prefix_len = len(input_ids[batch_id])
        possible_conts = []
        for allowed_id in allowed_ids:
            if len(allowed_id) > prefix_len:
                if input_ids[batch_id] == allowed_id[:prefix_len]:
                    possible_conts.append(allowed_id[prefix_len])
                    
        print(possible_conts)
        return possible_conts
    
    return prefix_constraining_fn

In [28]:
prefix_constraining_fn = get_prefix_allowed_tokens_fn(allowed_seqs)

[[2, 0, 713, 16, 5, 4195, 9, 127, 78, 586, 11, 230, 2], [2, 0, 3084, 117, 117, 42, 16, 47304, 2]]


In [29]:
generated_ids = model.generate(batch['input_ids'], 
                               pad_token_id=tokenizer.pad_token_id, 
                               eos_token_id=tokenizer.eos_token_id, #len(tokenizer) - 1, 
                               bos_token_id=tokenizer.bos_token_id,  
                               prefix_allowed_tokens_fn=prefix_constraining_fn,
#                                num_beams=2,
                              )

0
tensor([2])
[]
0
tensor([2])
[]
0
tensor([2])
[]
0
tensor([2])
[]
0
tensor([2, 5])
[]
0
tensor([2, 4])
[]
0
tensor([2, 6])
[]
0
tensor([2, 0])
[]
0
tensor([2, 5, 5])
[]
0
tensor([2, 5, 4])
[]
0
tensor([2, 5, 6])
[]
0
tensor([2, 5, 0])
[]
0
tensor([2, 5, 5, 5])
[]
0
tensor([2, 5, 5, 4])
[]
0
tensor([2, 5, 5, 6])
[]
0
tensor([2, 5, 5, 0])
[]


In [30]:
print(tokenizer.batch_decode(generated_ids))


['</s> the the the</s>']
